In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import models, transforms
from sklearn.model_selection import StratifiedKFold
from torchvision.models import resnet50, ResNet50_Weights
import time
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

In [3]:
# Define global parameters
PARAMS = {
    'width': 256,
    'height': 256,
    'B': 0.5,
    'split': 0.8,
    'epochs': 1,
    'learning_rate': 0.0001,
    'batch_size': 32,
    'folds': 3,  # Number of folds for stratified k-fold
    'optimizer': 'adam',  # 'adam', 'rmsprop', 'sgd', etc.
    'loss_function': 'BCE', # 'cross_entropy','BCE'
    'momentum': 0,  # Add momentum for SGD optimizer
    'model_architecture': 'resnet34',  # Change to any other model
}

In [4]:
# Load data
train_dir = '/content/drive/MyDrive/CV_Final_Project/isic_dataset/train/'
train_df = pd.read_csv('/content/drive/MyDrive/CV_Final_Project/isic_dataset/truth.csv')

In [5]:
train_df.shape[0]

33126

In [6]:
labels=[]
data=[]
for i in range(train_df.shape[0]):
    data.append(train_dir + train_df['image_name'].iloc[i]+'.jpg')
    labels.append(train_df['target'].iloc[i])
df=pd.DataFrame(data)
df.columns=['images']
df['target']=labels
df

,images,target
0,/content/drive/MyDrive/CV_Final_Project/isic_d...,0
1,/content/drive/MyDrive/CV_Final_Project/isic_d...,0
2,/content/drive/MyDrive/CV_Final_Project/isic_d...,0
3,/content/drive/MyDrive/CV_Final_Project/isic_d...,0
4,/content/drive/MyDrive/CV_Final_Project/isic_d...,0
...,...,...
33121,/content/drive/MyDrive/CV_Final_Project/isic_d...,0
33122,/content/drive/MyDrive/CV_Final_Project/isic_d...,0
33123,/content/drive/MyDrive/CV_Final_Project/isic_d...,0
33124,/content/drive/MyDrive/CV_Final_Project/isic_d...,0


In [7]:
# Define preprocessing transforms
preprocess_train = transforms.Compose([
    transforms.Resize((PARAMS['height'], PARAMS['width'])),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    #transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), #Resnet50 Normalization
])

preprocess_val = transforms.Compose([
    transforms.Resize((PARAMS['height'], PARAMS['width'])),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [8]:
class ImageDataset(Dataset):
    def __init__(self, data_paths, labels, transform=None, mode='train'):
        self.data = data_paths
        self.labels = labels
        self.transform = transform
        self.mode = mode

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = self.data[idx]
        img = cv2.imread(img_name)
        # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)

        if self.transform is not None:
            img = self.transform(img)

        if self.mode == 'test':
            return img
        else:
            labels = torch.tensor(self.labels[idx])

            return img, labels

In [9]:
class ResNet34(nn.Module):
    def __init__(self):
        super(ResNet34, self).__init__()
        self.model = models.resnet34(pretrained=True)  # Load pre-trained ResNet50
        num_features = self.model.fc.in_features
        self.model.fc = nn.Linear(num_features, 1)  # Change the output to 1 for binary classification

    def forward(self, x):
        return self.model(x)

In [10]:
def get_optimizer_and_criterion(PARAMS, model):
    # Update the loss function based on the PARAMS
    if PARAMS['loss_function'] == 'cross_entropy':
        criterion = nn.CrossEntropyLoss()
    elif PARAMS['loss_function'] == 'BCE':
        criterion = nn.BCEWithLogitsLoss()
    # Add more conditions for other loss functions as needed
    else:
        raise ValueError(f"Unsupported loss function: {PARAMS['loss_function']}")

    # Update the optimizer based on the PARAMS
    if PARAMS['optimizer'] == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=PARAMS['learning_rate'])
    elif PARAMS['optimizer'] == 'rmsprop':
        optimizer = optim.RMSprop(model.parameters(), lr=PARAMS['learning_rate'])
    elif PARAMS['optimizer'] == 'sgd':
        optimizer = optim.SGD(model.parameters(), lr=PARAMS['learning_rate'], momentum=PARAMS['momentum'])
    else:
        raise ValueError(f"Unsupported optimizer: {PARAMS['optimizer']}")

    return optimizer, criterion

In [11]:
def train_model(train_loader, val_loader, model, optimizer, criterion, device, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0
        start_time = time.time()
        print(f'Epoch [{epoch + 1}/{num_epochs}] Training...')

        print(len(train_loader))
        counter = 0

        for inputs, labels in train_loader:
            print("Counter:", counter)
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels.float().unsqueeze(1))
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            # Calculate accuracy
            predicted = (torch.sigmoid(outputs) >= 0.5).flatten().cpu().numpy()
            total_train += labels.size(0)
            correct_train += (predicted == labels.cpu().numpy()).sum().item()

            counter += 1

        counter = 0

        # Print training loss and accuracy at the end of each epoch
        end_time = time.time()  # Record the end time for the epoch
        epoch_time = end_time - start_time  # Calculate the time taken for the epoch
        average_loss = running_loss / len(train_loader)
        train_accuracy = correct_train / total_train
        # Log training accuracy and loss to WandB
        print(f'Training Loss: {average_loss}, Training Accuracy: {train_accuracy}, Time: {epoch_time} seconds')


        # Evaluate the model on the validation set
        model.eval()
        correct_val = 0
        total_val = 0
        val_running_loss = 0.0
        start_val_time = time.time()
        y_true = []
        y_pred = []
        with torch.no_grad():
            for inputs, labels in val_loader:
                print("Counter:", counter)
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                val_loss = criterion(outputs, labels.float().unsqueeze(1))
                val_running_loss += val_loss.item()

                # Calculate validation accuracy
                predicted = (torch.sigmoid(outputs) >= 0.5).flatten().cpu().numpy()
                y_true.extend(labels.cpu().numpy())
                y_pred.extend(predicted)
                total_val += labels.size(0)
                correct_val += (predicted == labels.cpu().numpy()).sum().item()

                counter += 1

        # Print validation loss and accuracy
        validation_loss = val_running_loss / len(val_loader)
        validation_accuracy = correct_val / total_val
        conf_matrix = confusion_matrix(y_true, y_pred)
        precision = precision_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)
        end_val_time = time.time()  # Record the end time for validation
        val_epoch_time = end_val_time - start_val_time
        # Log Validation accuracy and loss to WandB
        print(f'Validation Loss: {validation_loss}, Validation Accuracy: {validation_accuracy}, Validation Time: {val_epoch_time} seconds')
        # Assuming y_true contains the true labels and y_pred contains the predicted labels
        print(f'Confusion Matrix: {conf_matrix}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}')

In [12]:
# def test_model(test_loader, model, device):
#     model.eval()
#     predictions = []
#     image_names = []

#     image_names = test_df['image_name'].tolist()
#     test_start_time = time.time()
#     with torch.no_grad():
#         for i, data in enumerate(test_loader):
#             data = data.to(device)
#             outputs = model(data)

#             # Generate predictions for submission using sigmoid
#             probabilities = (torch.sigmoid(outputs) >= 0.5).cpu().numpy()
#             for prob in probabilities:
#                 predictions.append(int(prob.item()))

#     test_end_time = time.time()
#     test_time = test_end_time - test_start_time
#     # Save predictions to CSV file
#     submission_df = pd.DataFrame({'image_name': image_names, 'target': predictions})
#     #submission_df = pd.DataFrame({'id': range(1, len(predictions) + 1), 'target': predictions})
#     submission_df.to_csv('submission.csv', index=False)
#     print(f'Test Evaluation and Submission CSV is generated in : {test_time} seconds')

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

# Initialize model & optimizer
model = ResNet34().to(device)
optimizer,criterion = get_optimizer_and_criterion(PARAMS,model)
print(model)

Device: cuda


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 147MB/s]


ResNet34(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runn

In [15]:
from tqdm import tqdm

# Implement stratified k-fold cross-validation
skf = StratifiedKFold(n_splits=PARAMS['folds'], shuffle=True, random_state=42)
for fold, (train_index, val_index) in tqdm(enumerate(skf.split(train_df['image_name'], train_df['target']))):
    print(f'Fold {fold + 1}')
    # --- Read in Data ---
    train_data = df.iloc[train_index].reset_index(drop=True)
    valid_data = df.iloc[val_index].reset_index(drop=True)

    # Create datasets & data loaders
    train_dataset=ImageDataset(data_paths=train_data['images'].values, labels=train_data['target'].values, transform=preprocess_train)
    train_subset_indices = np.random.choice(len(train_dataset), size=int(0.05 * len(train_dataset)), replace=False)
    train_subset = Subset(train_dataset, train_subset_indices)
    print(len(train_subset))
    val_dataset=ImageDataset(data_paths=valid_data['images'].values, labels=train_data['target'].values, transform=preprocess_val)
    val_subset_indices = np.random.choice(len(val_dataset), size=int(0.05 * len(val_dataset)), replace=False)
    val_subset = Subset(val_dataset, val_subset_indices)
    print(len(val_subset))

    train_loader = DataLoader(train_subset, batch_size=PARAMS['batch_size'], shuffle=True)
    print(len(train_loader))
    val_loader = DataLoader(val_subset, batch_size=PARAMS['batch_size'], shuffle=False)

    train_model(train_loader, val_loader, model, optimizer, criterion, device, PARAMS['epochs'])

print('Training completed.')

0it [00:00, ?it/s]

Fold 1
1104
552
35
Epoch [1/1] Training...
35
Counter: 0
Counter: 1
Counter: 2
Counter: 3
Counter: 4
Counter: 5
Counter: 6
Counter: 7
Counter: 8
Counter: 9
Counter: 10
Counter: 11
Counter: 12
Counter: 13
Counter: 14
Counter: 15
Counter: 16
Counter: 17
Counter: 18
Counter: 19
Counter: 20
Counter: 21
Counter: 22
Counter: 23
Counter: 24
Counter: 25
Counter: 26
Counter: 27
Counter: 29
Counter: 30
Counter: 31
Counter: 32
Counter: 33
Counter: 34
Training Loss: 0.2541838922670909, Training Accuracy: 0.927536231884058, Time: 969.4265472888947 seconds
Counter: 0
Counter: 1
Counter: 2
Counter: 3
Counter: 4
Counter: 5
Counter: 6
Counter: 7
Counter: 8
Counter: 9
Counter: 10
Counter: 11
Counter: 12
Counter: 13
Counter: 14
Counter: 15
Counter: 16


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
1it [23:30, 1410.26s/it]

Counter: 17
Validation Loss: 0.09056631323053604, Validation Accuracy: 0.9873188405797102, Validation Time: 440.8192286491394 seconds
Confusion Matrix: [[545   0]
 [  7   0]], Precision: 0.0, Recall: 0.0, F1 Score: 0.0
Fold 2
1104
552
35
Epoch [1/1] Training...
35
Counter: 0
Counter: 1
Counter: 2
Counter: 3
Counter: 4
Counter: 5
Counter: 6
Counter: 7
Counter: 8
Counter: 9
Counter: 10
Counter: 11
Counter: 12
Counter: 13
Counter: 14
Counter: 15
Counter: 16
Counter: 17
Counter: 18
Counter: 19
Counter: 20
Counter: 21
Counter: 22
Counter: 23
Counter: 24
Counter: 25
Counter: 26
Counter: 27
Counter: 28
Counter: 29
Counter: 30
Counter: 31
Counter: 32
Counter: 33
Counter: 34
Training Loss: 0.07614072183413165, Training Accuracy: 0.9864130434782609, Time: 906.7725872993469 seconds
Counter: 0
Counter: 1
Counter: 2
Counter: 3
Counter: 4
Counter: 5
Counter: 6
Counter: 7
Counter: 8
Counter: 9
Counter: 10
Counter: 11
Counter: 12
Counter: 13
Counter: 14
Counter: 15
Counter: 16


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
2it [45:48, 1367.61s/it]

Counter: 17
Validation Loss: 0.11002005507341689, Validation Accuracy: 0.9782608695652174, Validation Time: 430.980229139328 seconds
Confusion Matrix: [[540   0]
 [ 12   0]], Precision: 0.0, Recall: 0.0, F1 Score: 0.0
Fold 3
1104
552
35
Epoch [1/1] Training...
35
Counter: 0
Counter: 1
Counter: 2
Counter: 3
Counter: 4
Counter: 5
Counter: 6
Counter: 7
Counter: 8
Counter: 9
Counter: 10
Counter: 11
Counter: 12
Counter: 13
Counter: 14
Counter: 15
Counter: 16
Counter: 17
Counter: 18
Counter: 19
Counter: 20
Counter: 21
Counter: 22
Counter: 23
Counter: 24
Counter: 25
Counter: 26
Counter: 27
Counter: 28
Counter: 29
Counter: 30
Counter: 31
Counter: 32
Counter: 33
Counter: 34
Training Loss: 0.09231451403881823, Training Accuracy: 0.9827898550724637, Time: 858.4612669944763 seconds
Counter: 0
Counter: 1
Counter: 2
Counter: 3
Counter: 4
Counter: 5
Counter: 6
Counter: 7
Counter: 8
Counter: 9
Counter: 10
Counter: 11
Counter: 12
Counter: 13
Counter: 14
Counter: 15
Counter: 16


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
3it [1:07:08, 1342.89s/it]

Counter: 17
Validation Loss: 0.07507905198468103, Validation Accuracy: 0.9873188405797102, Validation Time: 422.16232347488403 seconds
Confusion Matrix: [[545   0]
 [  7   0]], Precision: 0.0, Recall: 0.0, F1 Score: 0.0
Training completed.


In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/CV_Final_Project/resnet34_baseline.pt')